In [52]:
from pylab import *
import bisect
def unit(i,n):
    result = zeros(n)
    result[i] = 1.0
    return result

# Generating a Dataset

In [10]:
M = roll(diag(ones(4)) + 0.1*rand(4,4),-1,1)
M /= sum(M,axis=0)[newaxis,:]

In [55]:
def generate(M,s=0,n=10):
    result = []
    for i in range(n):
        result.append(s)
        x = unit(s,len(M))
        x = dot(M,x)
        s = bisect.bisect_left(add.accumulate(x),rand())
    return result

In [63]:
def glabel(s):
    result = [0]
    for i in range(1,len(s)):
        if s[i]==s[i-1]+1 and result[-1]:
            result.append(s[i])
        else:
            result.append(0)
    return result

In [78]:
xs = generate(M,0,10000)
ys = glabel(xs)

In [110]:
print xs[:20]
print ys[:20]

[0, 1, 2, 3, 1, 2, 3, 0, 1, 2, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]
[0, 1, 2, 3, 0, 2, 3, 0, 1, 2, 0, 3, 0, 1, 2, 3, 0, 1, 2, 3]


# Learning the Transitions

In [87]:
cs = []
ps = []
for i in range(1,len(xs)):
    c = concatenate([unit(ys[i-1],4),unit(xs[i],4)])
    p = unit(xs[i],4)
    cs.append(c)
    ps.append(p)

In [88]:
from sklearn.linear_model import LogisticRegression

In [89]:
lr = LogisticRegression()
lr.fit(cs,ys[1:])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', tol=0.0001)

In [103]:
sum(array(lr.predict(cs),'i')!=array(ys[1:],'i'))

442

# Forward Algorithm

In [105]:
pys = [0]
for i in range(1,len(xs)):
    c = concatenate([unit(pys[i-1],4),unit(xs[i],4)])
    y = lr.predict([c])
    pys.append(y)

In [108]:
sum(array(ys)!=array(pys))

382

In [91]:
array(lr.coef_*10,'i')

array([[-11, -18, -10,  48,  49, -15,  -8, -17],
       [ 36, -31, -28, -30, -31,  47, -34, -34],
       [ -2,  41, -44, -37, -28, -35,  51, -31],
       [ -2, -40,  40, -40, -21, -34, -37,  51]], dtype=int32)

In [111]:
lprobs = zeros((100,4))

In [112]:
lprobs[0,:] = log(1.0/4)